# Direct

This notebook was automatically generated from the Algorithms for Optimization source code. Each cell generates a figure from the original text. While this code is not optimized for use in lectures, we provide it here to be adapted for such projects. We hope you find it useful.

In [ ]:
include("support_code.jl");

In [ ]:
using Optim

p = let

	f = x -> x[1]^2 - 0.9*x[1]*x[2] + x[2]^2

	xdomain = (-2, 1)
	ydomain = (-2, 1)

	coordinate = 1

	x0 = [-1.5, -1.5]
	pts = Vector{Float64}[x0]
	dir = zeros(Float64, length(x0))
	for i in 1 : 10
		x = pts[end]
		dir[coordinate] = 1.0
		α = optimize(α->f(x + α*dir), -100.0, 100.0).minimizer
		push!(pts, x + α*dir)
		dir[coordinate] = 0.0
		coordinate = mod(coordinate,length(x))+1
	end

	plots = Plots.Plot[]
	push!(plots, Plots.Contour(f, xdomain, ydomain, levels=[0.1,0.2,0.5,1,2,3,4], style="width=\\columnwidth"))
	push!(plots, Plots.Linear3([x[1] for x in pts], [x[2] for x in pts], [f(x) for x in pts], style="black, solid, thick, mark=none"))
	Axis(plots, width="1.1*9cm", height="1.1*9cm", xlabel=L"x_1", ylabel=L"x_2", style="xtick=\\empty, ytick=\\empty, contour/labels=false, axis equal, view={0}{90}")
end

plot(p)

In [ ]:

    p = let
        f = x -> abs(x[1]+x[2]) + abs(x[2] - x[1]) - 7exp(-(x[1]-1.5)^2 - (x[2]-1.5)^2)*(x[1]+x[2])

        xdomain = (-1, 3)
        ydomain = (-1, 3)

        plots = Plots.Plot[]
        push!(plots, Plots.Contour(f, xdomain, ydomain, levels=[-15,-10,-5,-1,0,0.5,1,1.5,2,2.5,3], style="width=\\columnwidth"))
        push!(plots, Plots.Linear3([0,0,0.5], [0.5,0,0], [f([0,0.5]), f([0,0]), f([0.5,0])], style="black, solid, mark=none"))
        Axis(plots, width="1.1*9cm", height="1.1*9cm", xlabel=L"x_1", ylabel=L"x_2", style="xtick=\\empty, ytick=\\empty, contour/labels=false, axis equal, view={0}{90}")
    end

    plot(p)

In [ ]:
using Optim
p = let
	function basis(i, n)
	    eᵢ = zeros(n)
	    eᵢ[i] = 1
	    eᵢ
	end

	f = x -> x[1]^2 - 0.9*x[1]*x[2] + x[2]^2

	xdomain = (-2, 1)
	ydomain = (-2, 1)


	x0 = [-1.5, -1.5]
	pts = Vector{Float64}[x0]

	n = length(x0)
	U = [basis(i,n) for i in 1 : n]

	for i in 1 : 3
		x = pts[end]
		for i in 1 : n
	        dir = U[i]
	        α = optimize(α->f(x + α*dir), -100.0, 100.0).minimizer
	        x += α*dir
	        push!(pts, x)
	    end
	    for i in 1 : n-1
	        U[i] = U[i+1]
	    end
	    U[n] = dir = x - x0
	    α = optimize(α->f(x + α*dir), -100.0, 100.0).minimizer
	    x0 = x + α*dir
	    push!(pts, x0)
	end


	plots = Plots.Plot[]
	push!(plots, Plots.Contour(f, xdomain, ydomain, levels=[0.1,0.2,0.5,1,2,3,4], style="width=\\columnwidth"))
	push!(plots, Plots.Linear3([x[1] for x in pts], [x[2] for x in pts], [f(x) for x in pts], style="black, thick, solid, mark=none"))
	Axis(plots, width="1.1*9cm", height="1.1*9cm", xlabel=L"x_1", ylabel=L"x_2", style="xtick=\\empty, ytick=\\empty, contour/labels=false, axis equal, view={0}{90}")
end

plot(p)

In [ ]:

	p = let
		g = GroupPlot(4,1,groupStyle="xlabels at=edge bottom, ylabels at=edge left, horizontal sep=0.5cm")

		f = x -> -exp(-(x[1]*x[2] - 1.5)^2 -(x[2]-1.5)^2)

		basis(i, n) = [k == i ? 1. : 0. for k in 1 : n]

		xdomain = (0, 3)
		ydomain = (0, 3)

		p_contour = Plots.Contour(f, xdomain, ydomain, style="width=\\columnwidth", xbins=151, ybins=151) # , levels=[-15,-10,-5,-1,0,0.5,1,1.5,2,2.5,3]

		function add_hooke_jeeves_pts!(plots, x, α)
			push!(plots, Plots.Linear3([x[1] - α,x[1] + α,x[1],x[1]], [x[2],x[2],x[2]-α, x[2]+α], [f([x[1]-α,x[2]]), f([x[1]+α,x[2]]), f([x[1],x[2]-α]), f([x[1],x[2]+α])], style="only marks, mark=*, mark size=1, mark options={draw=black, fill=black}"))
			push!(plots, Plots.Linear3([x[1]], [x[2]], [f([x[1],x[2]])], style="only marks, mark=*, mark size=1.75, mark options={draw=black, fill=black}"))
		end

		x = [0.75, 0.75]
		α = 0.5
		γ = 0.5
		y, n = f(x), length(x)
		for i in 1 : 4

			plots = Plots.Plot[]
			push!(plots, p_contour)
			add_hooke_jeeves_pts!(plots, x, α)
			ax = Axis(plots, ymin=0, xmin=0, ymax=3, xmax=3, width="5cm", height="5cm", xlabel=L"x_1", ylabel=L"x_2", style="xtick=\\empty, ytick=\\empty, contour/labels=false, axis equal, view={0}{90}")
			push!(g, ax)

	        improved = false
	        x_best, y_best = x, y
	        for i in 1 : n
            	for sgn in (-1,1)
    	            x′ = x + sgn*α*basis(i, n)
    	            y′ = f(x′)
    	            if y′ < y_best
    	                x_best, y_best, improved = x′, y′, true
    	            end
                end
	        end
            x, y = x_best, y_best

	        if !improved
	            α *= γ
	        end
	    end
		g
	end

	plot(p)

In [ ]:
using DataStructures
struct Interval
    c
    y
    depths
end
const Intervals = OrderedDict{Float64,PriorityQueue{Interval, Float64}}

p = let

	f = x -> sin(x) + sin(2x) + sin(4x) + sin(8x)

	basis(i, n) = [k == i ? 1. : 0. for k in 1 : n]

	rev_unit_hypercube_parameterization(x, a, b) = x.*(b-a) + a
	function reparameterize_to_unit_hypercube(f, a, b)
	    Δ = b-a
	    return x->f(x.*Δ + a)
	end

	min_depth(interval) = minimum(interval.depths)
	vertex_dist(interval) = norm(0.5*3.0.^(-interval.depths), 2)

	function add_interval!(intervals, interval)
		d = min_depth(interval)
	    if !haskey(intervals, d)
	        intervals[d] = PriorityQueue{Interval, Float64}()
	    end
	    enqueue!(intervals[d], interval, interval.y)
	end

	function get_opt_intervals(intervals, ϵ, y_best)
	    stack = Interval[]
	    for (d, pq) in intervals
	    	if !isempty(pq)
	            interval = DataStructures.peek(pq)[1]
	            x, y = d, interval.y
	            while !isempty(stack)
	            	interval1 = stack[end]
	            	x1, y1 = vertex_dist(interval1), interval1.y
	            	L1 = (y - y1)/(x - x1)
	            	if y1 - L1*x1 > y_best - ϵ || y < y1
	            		pop!(stack)
	            	elseif length(stack) > 1
	            		interval2 = stack[end-1]
	            		x2, y2 = vertex_dist(interval2), interval2.y
	            		L2 = (y1 - y2)/(x1 - x2)
	            		if L2 > L1
	            			pop!(stack)
	                    else
	                        break
	            		end
	                else
	                    break
	            	end
	           	end
	            push!(stack, interval) # add new interval
	        end
	    end
	    stack
	end

	function divide(f, interval)
	    c, d, n = interval.c, min_depth(interval), length(interval.c)
	    dirs = findall(interval.depths .== d)
	    cs = [(c + 3.0^(-d-1)*basis(i,n),
	           c - 3.0^(-d-1)*basis(i,n)) for i in dirs]
	    vs = [(f(C[1]), f(C[2])) for C in cs]
	    minvals = [min(V[1], V[2]) for V in vs]

	    retval = Interval[]
	    depths = copy(interval.depths)
	    for j in sortperm(minvals)
	        depths[dirs[j]] += 1
	        C, V = cs[j], vs[j]
	        push!(retval, Interval(C[1], V[1], copy(depths)))
	        push!(retval, Interval(C[2], V[2], copy(depths)))
	    end
	    push!(retval, Interval(c, interval.y, copy(depths)))
	    retval
	end

	function add_axes!(g, intervals, opt_intervals)

	    x_pts = collect(range(-2,stop=2,length=200))
	    y_pts = f.(x_pts)

	    style_normal = "clip marker paths, solid, only marks, mark=*, mark size=1, mark options={draw=black, fill=black}"
	    style_opt = "clip marker paths, solid, only marks, mark=*, mark size=1.25, mark options={draw=pastelBlue, fill=pastelBlue}"

	    arr_x_normal = Float64[]
	    arr_y_normal = Float64[]
	    arr_h_normal = Float64[]
	    arr_x_opt = Float64[]
	    arr_y_opt = Float64[]
	    arr_h_opt = Float64[]

	    plot_intervals = Plots.Plot[]
	    push!(plot_intervals, Plots.Linear(x_pts, y_pts, style="black, solid, mark=none"))
	    for (d,Is) in intervals
	        for I in keys(Is)
	            x = rev_unit_hypercube_parameterization(I.c, a, b)[1]
	            y = I.y
	            h = 2*3.0^(-d)
	            if in(I, opt_intervals)
	                push!(arr_x_opt, x)
	                push!(arr_y_opt, y)
	                push!(arr_h_opt, h)
	                push!(plot_intervals, Plots.Linear([x-h,x+h],[y,y], style="solid, pastelBlue, mark=none"))
	            else
	                push!(arr_x_normal, x)
	                push!(arr_y_normal, y)
	                push!(arr_h_normal, h)
	                push!(plot_intervals, Plots.Linear([x-h,x+h],[y,y], style="solid, gray, mark=none"))
	            end
	        end
	    end

	    push!(plot_intervals, Plots.Scatter(arr_x_normal, arr_y_normal, style=style_normal))
	    push!(plot_intervals, Plots.Scatter(arr_x_opt, arr_y_opt, style=style_opt))

	    push!(g, Axis(plot_intervals, xmin=-2, xmax=2, ymin=-2.5, ymax=2.5, ylabel=L"y"))

	    p_scatter_normal = Plots.Scatter(arr_h_normal, arr_y_normal, style=style_normal)
	    p_scatter_opt = Plots.Scatter(arr_h_opt, arr_y_opt, style=style_opt)

	    push!(g, Axis([p_scatter_normal, p_scatter_opt], xmin=0.0, xmax=2.0, ymin=-2.5, ymax=2.5))

	    g
	end

	ϵ = 0.001
	a = [-2.0]
	b = [2.0]
	g = reparameterize_to_unit_hypercube(x->f(x[1]), a, b)
	intervals = Intervals()
	n = length(a)
	c = fill(0.5, n)
	interval = Interval(c, g(c), fill(0, n))
	add_interval!(intervals, interval)
	c_best, y_best = copy(interval.c), interval.y

	K = 6
	G = GroupPlot(2, K, groupStyle="horizontal sep=0.25cm, vertical sep=0.25cm, xlabels at=edge bottom, xticklabels at=edge bottom, ylabels at=edge left, yticklabels at=edge left", style="width=6cm, height=3.25cm")

	for k in 1 : K
	    S = get_opt_intervals(intervals, ϵ, y_best)

	    add_axes!(G, intervals, get_opt_intervals(intervals, ϵ, y_best))

	    to_add = Interval[]
	    for interval in S
	        append!(to_add, divide(g, interval))
	        dequeue!(intervals[min_depth(interval)])
	    end
	    for interval in to_add
	        add_interval!(intervals, interval)
	        if interval.y < y_best
	            c_best, y_best = copy(interval.c), interval.y
	        end
	    end
	end

	G.axes[end-1].xlabel=L"x"
	G.axes[end].xlabel="interval half-width"
	G
end

plot(p)